### 강의에서 사용된 파이썬 주요 기능

- 이벤트 위치 정적 시각화
  - matplotlib.pyplot.scatter: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.scatter.html
  - matplotlib.pyplot.arrow: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.arrow.html
  - matplotlib.pyplot.savefig: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.savefig.html

- 이벤트 위치 반응형 시각화
  - plotly.graph_objects.Scatter: https://plotly.com/python-api-reference/generated/plotly.graph_objects.Scatter.html
  - plotly.graph_objects.Figure: https://plotly.com/python-api-reference/generated/plotly.graph_objects.Figure.html

- 이벤트 히트맵 시각화
  - pandas.concat: https://pandas.pydata.org/docs/reference/api/pandas.concat.html
  - numpy.histogram2d: https://numpy.org/doc/stable/reference/generated/numpy.histogram2d.html
  - matplotlib.pyplot.imshow: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html
  - matplotlib.pyplot.colorbar: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.colorbar.html
  - matplotlib colormaps: https://matplotlib.org/stable/tutorials/colors/colormaps.html
  - matplotlib.axes.Axes.tick_params: https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.tick_params.html

### 경기 데이터 불러오기

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from src.plot_utils import *

In [ ]:
match_id = 2057988
match_events = pd.read_pickle(f'data/refined_events/World_Cup/{match_id}.pkl')
match_events

### 이벤트 위치 정적 시각화(static plotting)

##### (1) plt.scatter 함수를 활용한 이벤트 위치 시각화

In [ ]:
team1_name, team2_name = match_events['team_name'].unique()
team1_events = match_events[match_events['team_name'] == team1_name]
team2_events = match_events[match_events['team_name'] == team2_name]

plt.scatter(team1_events['start_x'], team1_events['start_y'], c='blue')
plt.scatter(team2_events['start_x'], team2_events['start_y'], c='red')
plt.show()

##### (2) 공개 코드를 이용한 경기장 이미지 시각화

In [ ]:
draw_pitch(pitch='white', line='black')

##### (3) 경기장 이미지 위에 이벤트 발생 위치 시각화 및 꾸미기

In [ ]:
team2_events[['start_x', 'end_x']] = 104 - team2_events[['start_x', 'end_x']]
team2_events[['start_y', 'end_y']] = 68 - team2_events[['start_y', 'end_y']]

In [ ]:
draw_pitch('white', 'black')

plt.scatter(
    team1_events['start_x'], team1_events['start_y'], c='blue', edgecolors='k', alpha=0.7,
    label=f'{team1_name}: {len(team1_events)} events'
)
plt.scatter(
    team2_events['start_x'], team2_events['start_y'], marker='s', c='red', edgecolors='k', alpha=0.7,
    label=f'{team2_name}: {len(team2_events)} events'
)
plt.legend(fontsize=20, bbox_to_anchor=(1, 1))

# plt.savefig('img/event_scatter.png', bbox_inches='tight')
plt.show()

##### (4) plt.arrow 함수를 활용한 패스 경로 시각화

In [ ]:
pass_records = match_events[
    (match_events['event_type'] == 'Pass') |
    (match_events['sub_event_type'].isin(['Free kick', 'Free kick cross', 'corner']))
]
team1_pass_records = pass_records[pass_records['team_name'] == team1_name]
team2_pass_records = pass_records[pass_records['team_name'] == team2_name]

draw_pitch('white', 'black')

plt.scatter(
    team1_pass_records['start_x'], team1_pass_records['start_y'], marker='s', c='blue', alpha=0.7,
    label=f'{team1_name}: {len(team1_pass_records)} passes'
)
plt.scatter(
    team2_pass_records['start_x'], team2_pass_records['start_y'], marker='s', c='red', alpha=0.7,
    label=f'{team2_name}: {len(team2_pass_records)} passes'
)

for i, record in pass_records.iterrows():
    x = record['start_x']
    y = record['start_y']
    dx = record['end_x'] - x
    dy = record['end_y'] - y
    color = 'blue' if record['team_name'] == team1_name else 'red'
    plt.arrow(x, y, dx, dy, width=0.3, head_width=1.5, color=color, alpha=0.5)

plt.legend(fontsize=20, bbox_to_anchor=(1, 1))

# plt.savefig('img/pass_arrow.png', bbox_inches='tight')
plt.show()

### 이벤트 위치 반응형 시각화(interactive plotting)

##### (1) go.Scatter 클래스를 활용한 이벤트 위치 반응형 시각화

In [ ]:
match_events['display_time'] = match_events.apply(
    lambda x: f"{x['period']} {int(x['time'] // 60):02d}:{int(x['time'] % 60):02d}", axis=1
)
match_events

In [ ]:
match_title = f'{team1_name} - {team2_name}'
label_func = lambda x: f"{x['event_type']} by {x['player_name']}, {x['display_time']}"

team1_events = match_events[match_events['team_name'] == team1_name]
team1_trace = go.Scatter(
    x=team1_events['start_x'],
    y=team1_events['start_y'],
    name=team1_name,
    text=team1_events.apply(label_func, axis=1),
    mode='markers',
    marker=dict(size=8, color='blue')
)

team2_events = match_events[match_events['team_name'] == team2_name]
team2_events[['start_x', 'end_x']] = 104 - team2_events[['start_x', 'end_x']]
team2_events[['start_y', 'end_y']] = 68 - team2_events[['start_y', 'end_y']]
team2_trace = go.Scatter(
    x=team2_events['start_x'],
    y=team2_events['start_y'],
    name=team2_name,
    text=team2_events.apply(label_func, axis=1),
    mode='markers',
    marker=dict(size=8, color='red', symbol='square')
)

fig = go.Figure(data=[team1_trace, team2_trace], layout=get_pitch_layout(match_title))
fig.show()

##### (2) 이벤트 위치 반응형 시각화 함수 구현

- 시각화 함수 정의

In [ ]:
def plot_events(events, col_name, group_dict, event_type='all', rotate_team2_events=False):
    if event_type == 'all':
        match_title = f'{team1_name} - {team2_name}'
    else:
        events = events[events['event_type'] == event_type]
        match_title = f'{team1_name} - {team2_name} ({event_type})'

    if rotate_team2_events:
        events = events.copy()
        team2_idx = events['team_name'] == team2_name
        events.loc[team2_idx, ['start_x', 'end_x']] = 104 - events.loc[team2_idx, ['start_x', 'end_x']]
        events.loc[team2_idx, ['start_y', 'end_y']] = 68 - events.loc[team2_idx, ['start_y', 'end_y']]

    label_func = lambda x: f"{x['event_type']} by {x['player_name']}, {x['display_time']}"
    trace_list = []

    for group_name, color in group_dict.items():
        group_events = events[events[col_name] == group_name]
        trace = go.Scatter(
            x=group_events['start_x'],
            y=group_events['start_y'],
            text=group_events.apply(label_func, axis=1),
            mode='markers',
            marker=dict(size=8, color=color, symbol='square')
        )
        trace['name'] = group_name
        trace_list.append(trace)

    fig = go.Figure(data=trace_list, layout=get_pitch_layout(match_title))
    fig.show()

- 양팀 슈팅 위치 시각화

In [ ]:
group_dict = {'Korea Republic': 'red', 'Germany': 'blue'}
plot_events(match_events, 'team_name', group_dict, event_type='Shot', rotate_team2_events=True)

- 주요 선수 패스 위치 시각화

In [ ]:
match_events['player_name'].unique()

In [ ]:
group_dict = {
    'Son Heung-Min': 'red', 'Jae-Sung Lee': 'hotpink',
    'M. Hummels': 'blue', 'T. Kroos': 'black'
}
plot_events(match_events, col_name='player_name', group_dict=group_dict, event_type='Pass')

### 이벤트 히트맵(heat map) 시각화

##### (1) 대회 전체 이벤트 데이터를 하나의 DataFrame으로 연결하기

In [ ]:
dataset_name = 'World_Cup'
match_df = pd.read_csv(f'data/refined_events/{dataset_name}/matches.csv', index_col=0, encoding='utf-8-sig')
match_df

In [ ]:
match_events_list = []

for match_id in match_df.index:
    match_events = pd.read_pickle(f'data/refined_events/World_Cup/{match_id}.pkl')
    match_events_list.append(match_events)

events = pd.concat(match_events_list, ignore_index=True)
events

##### (2) 선수별 이벤트 히트맵 그리기

- numpy.histogram2d 함수를 활용한 히트맵 산출

In [ ]:
player_events = events[events['player_name'] == 'Son Heung-Min']
x = player_events['start_x']
y = player_events['start_y']
heatmap, xedges, yedges = np.histogram2d(y, x, bins=(3, 6), range=[[0, 68], [0, 104]])
heatmap, xedges, yedges

- plt.imshow 함수를 활용한 히트맵 시각화

In [ ]:
plt.figure(figsize=(10, 6))
plt.imshow(heatmap)

- 경기장 위에 히트맵 오버레이(overlay)

In [ ]:
fig, ax = draw_pitch('white', 'black')

img = ax.imshow(heatmap[::-1], extent=[0, 104, 0, 68], vmin=0, cmap='Reds', alpha=0.8)
cbar = fig.colorbar(img, ax=ax)
cbar.ax.tick_params(labelsize=15)
cbar.set_label(label='Number of events', size=20)

# plt.savefig('img/player_event_heatmap.png', bbox_inches='tight')

##### (3) 대회 전체 슈팅 히트맵 그리기

In [ ]:
shot_records = events[events['event_type'] == 'Shot']
x = shot_records['start_x']
y = shot_records['start_y']
heatmap, xedges, yedges = np.histogram2d(y, x, bins=(34, 52), range=[[0, 68], [0, 104]])

fig, ax = draw_pitch('white', 'black')
img = ax.imshow(heatmap[::-1], extent=[0, 104, 0, 68], vmin=0, vmax=15, cmap='jet', alpha=0.8)

cbar = fig.colorbar(img, ax=ax)
cbar.ax.tick_params(labelsize=15)
cbar.set_label(label='Number of shots', size=20)

plt.savefig('img/shot_heatmap.png', bbox_inches='tight')